In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [975 kB]
Hit:12 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:13 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,59

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-04-16 17:26:01--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.42MB/s    in 0.2s    

2023-04-16 17:26:01 (5.42 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [21]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12190288|R3FU16928EP5TC|B00AYB1482|     668895143|Enlightened: Seas...|Digital_Video_Dow...|          5|            0|          0|   N|                Y|I loved it and I ...|I loved it and I ...|2015-08-31 00:00:00|
|         US|   30549954|R1IZHHS1MH3AQ4|B00KQD28OM|     246219280|      

### Create DataFrames to match tables

In [18]:
from pyspark.sql.functions import to_date
from pyspark.sql.functions import sum,avg,max,count
# Read in the Review dataset as a DataFrame
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [19]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg(count("*").alias("customer_count"))
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   36427874|             1|
|   17687096|            10|
|   43798350|             4|
|   49387236|             1|
|   24321836|             2|
|   42543176|             1|
|   16298288|             4|
|   12333316|             1|
|   25790304|             1|
|   13786319|             1|
|   24796052|             1|
|   53049917|             4|
|   30478898|             1|
|   20829560|             3|
|   29699080|             3|
|    8681732|             3|
|   34220092|             2|
|   13727857|            14|
|   13480593|             3|
|   49935991|             1|
+-----------+--------------+
only showing top 20 rows



In [20]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00YKA8PX0|        The Basement|
|B004R10N6O|       Morning Glory|
|B006MYGL8S|   Deadwood Season 1|
|B008Y2X78S|Boardwalk Empire ...|
|B003D8XCOC|Eating Out 2: Slo...|
|B01489KEKY|Some Kind of Beau...|
|B00E3WN0TY|   Banshee: Season 1|
|B00DYQNC3E|Masterpiece: Eliz...|
|B00QROEQ7G|             Amnesia|
|B00QWUK1PQ|     Free the Nipple|
|B00ZETHCTE|Louis C.K.: Oh My...|
|B00MQOXI8Y|   The Expendables 3|
|B003E4AXJC|Best of NOVA Volu...|
|B00H6PG6J6|Lee Daniels' The ...|
|B00NFDFCOI|  Words and Pictures|
|B002WXLJ58|The Lord Of The R...|
|B00KGJNQZQ|Robin Williams: W...|
|B005LLSZNM|Sons Of Anarchy S...|
|B0030JBBYW| The Point of Origin|
|B0064J6RU8|Atlas Shrugged Pa...|
+----------+--------------------+
only showing top 20 rows



In [22]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3FU16928EP5TC|   12190288|B00AYB1482|     668895143| 2015-08-31|
|R1IZHHS1MH3AQ4|   30549954|B00KQD28OM|     246219280| 2015-08-31|
| R52R85WC6TIAH|   52895410|B01489L5LQ|     534732318| 2015-08-31|
| R7HOOYTVIB0DS|   27072354|B008LOVIIK|     239012694| 2015-08-31|
|R1XQ2N5CDOZGNX|   26939022|B0094LZMT0|     535858974| 2015-08-31|
|R1HCST57W334KN|    4772040|B0112OSOQE|      38517795| 2015-08-31|
|R32BUTYQS1ZJBQ|   12910040|B000NPE5SA|     373323715| 2015-08-31|
|  RH4SXPL4L9QU|   38805573|B00XWV4QXG|     633842417| 2015-08-31|
|R37INWIQA5YW8N|   37100714|B00X8UKOUK|     666093513| 2015-08-31|
|R18GSVAAS3N8GX|   41234409|B00OOKXTFU|     801680808| 2015-08-31|
|R2QCG0ZZOHYUOX|   24698954|B00URDJ600|     174981485| 2015-08-31|
|R2B1OCYMUVMS5C|   11956809|B009JUZ5I4|     654293980| 2015-08

In [23]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3FU16928EP5TC|          5|            0|          0|   N|                Y|
|R1IZHHS1MH3AQ4|          5|            0|          0|   N|                Y|
| R52R85WC6TIAH|          4|           17|         18|   N|                Y|
| R7HOOYTVIB0DS|          5|            0|          0|   N|                Y|
|R1XQ2N5CDOZGNX|          5|            0|          0|   N|                Y|
|R1HCST57W334KN|          5|            5|          6|   N|                Y|
|R32BUTYQS1ZJBQ|          4|            1|          1|   N|                Y|
|  RH4SXPL4L9QU|          3|            0|          0|   N|                Y|
|R37INWIQA5YW8N|          2|            0|          0|   N|                Y|
|R18GSVAAS3N8GX|          3|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [26]:
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://aws-challenge.c9tisna2dx0v.us-east-2.rds.amazonaws.com:5432/AWS_Challenge"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [27]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [28]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [29]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [30]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)